In [1]:
import pandas as pd
import numpy as np # linear algebra
from sklearn.preprocessing import LabelEncoder
# Replace 'path_to_dataset' with the actual path to your dataset file
df = pd.read_csv('embedding_FirstStructureL6.csv')
X_final= df

df2 = pd.read_csv('Dataset33new (1).csv')
y = df2['Brand']

label_encoder = LabelEncoder()


# Initialize LabelEncoder for the target variable y
label_encoder_y = LabelEncoder()
y_encoded = label_encoder_y.fit_transform(y)

In [2]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_encoded, test_size=0.2, random_state=42)

In [3]:
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import BorderlineSMOTE

smote = SMOTE(random_state=42)
X_train_resampled_smote, y_train_resampled_smote = smote.fit_resample(X_train, y_train)

borderline_smote = BorderlineSMOTE(random_state=42)
X_train_resampled_borderline, y_train_resampled_borderline = borderline_smote.fit_resample(X_train, y_train)


## BorutaShap SMOTE + Bordeline SMOTE
focus on Bordeline SMOTE only also can

RF

In [5]:
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import BorderlineSMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Initialize Borderline SMOTE

# Define the parameter grid for GridSearchCV
param_grid = {
    'max_depth': [2, 3, 4, 5, None],
    'n_estimators': [50, 100, 150, 200],
}

# Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=rf_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV with resampled data
grid_search.fit(X_train_resampled_borderline, y_train_resampled_borderline)

# Get the best model
best_rf_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_rf = best_rf_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_rf = accuracy_score(y_test, y_pred_best_rf)
classification_report_result_best_rf = classification_report(y_test, y_pred_best_rf, digits=3)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy (Best Random Forest with Borderline SMOTE): {accuracy_best_rf}")
print("Classification Report (Best Random Forest with Borderline SMOTE):\n", classification_report_result_best_rf)


Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best Parameters: {'max_depth': None, 'n_estimators': 200}
Accuracy (Best Random Forest with Borderline SMOTE): 0.9959912536443148
Classification Report (Best Random Forest with Borderline SMOTE):
               precision    recall  f1-score   support

           0      0.962     1.000     0.980        75
           1      0.984     1.000     0.992        60
           2      1.000     1.000     1.000       182
           3      1.000     1.000     1.000        37
           4      0.995     0.995     0.995       212
           5      1.000     1.000     1.000        77
           6      0.996     1.000     0.998       248
           7      1.000     1.000     1.000        41
           8      1.000     1.000     1.000        58
           9      1.000     0.988     0.994       167
          10      1.000     1.000     1.000        43
          11      1.000     0.929     0.963        28
          12      1.000     1.000     

In [6]:
import joblib

# Save the model to a file
joblib_file = "best_rf.joblib"
joblib.dump(best_rf_classifier, joblib_file)

['best_rf.joblib']

Gaussian NB

In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
# Define the parameter grid
param_grid = {
    'var_smoothing': [1e-9, 1.778e-7, 3.162e-5, 5.6e-3, 1]
}

# Initialize the GaussianNB classifier
gnb_classifier = GaussianNB()

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=gnb_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled_borderline, y_train_resampled_borderline)

# Get the best model
best_gnb_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_gnb = best_gnb_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_gnb = accuracy_score(y_test, y_pred_best_gnb)
classification_report_result_best_gnb = classification_report(y_test, y_pred_best_gnb, digits=3)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy (Best GaussianNB with SMOTE): {accuracy_best_gnb}")
print("Classification Report (Best GaussianNB with SMOTE):\n", classification_report_result_best_gnb)


Fitting 5 folds for each of 5 candidates, totalling 25 fits
Best Parameters: {'var_smoothing': 0.0056}
Accuracy (Best GaussianNB with SMOTE): 0.7018950437317785
Classification Report (Best GaussianNB with SMOTE):
               precision    recall  f1-score   support

           0      1.000     0.507     0.673        75
           1      0.420     0.700     0.525        60
           2      0.830     0.830     0.830       182
           3      1.000     0.270     0.426        37
           4      0.474     0.887     0.617       212
           5      1.000     0.091     0.167        77
           6      0.811     0.673     0.736       248
           7      1.000     1.000     1.000        41
           8      0.909     0.517     0.659        58
           9      0.788     0.647     0.711       167
          10      1.000     1.000     1.000        43
          11      0.526     0.357     0.426        28
          12      0.000     0.000     0.000        66
          13      0.960     0

In [8]:
# Save the model to a file
joblib_file = "best_gnb.joblib"
joblib.dump(best_gnb_classifier, joblib_file)

['best_gnb.joblib']

DT

In [9]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# Define the parameter grid
param_grid = {
    'max_depth': [2, 3, 4, 5, None]
}

# Initialize the Decision Tree classifier
dt_classifier = DecisionTreeClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=dt_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled_borderline, y_train_resampled_borderline)

# Get the best model
best_dt_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_dt = best_dt_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_dt = accuracy_score(y_test, y_pred_best_dt)
classification_report_result_best_dt = classification_report(y_test, y_pred_best_dt, digits=3)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy (Best Decision Tree with SMOTE): {accuracy_best_dt}")
print("Classification Report (Best Decision Tree with SMOTE):\n", classification_report_result_best_dt)


Fitting 5 folds for each of 5 candidates, totalling 25 fits
Best Parameters: {'max_depth': None}
Accuracy (Best Decision Tree with SMOTE): 0.9446064139941691
Classification Report (Best Decision Tree with SMOTE):
               precision    recall  f1-score   support

           0      0.887     0.947     0.916        75
           1      0.754     0.767     0.760        60
           2      0.932     0.978     0.954       182
           3      0.946     0.946     0.946        37
           4      0.971     0.948     0.959       212
           5      1.000     0.922     0.959        77
           6      0.948     0.960     0.954       248
           7      1.000     1.000     1.000        41
           8      0.965     0.948     0.957        58
           9      0.958     0.958     0.958       167
          10      0.955     0.977     0.966        43
          11      0.741     0.714     0.727        28
          12      0.971     1.000     0.985        66
          13      0.958     0

In [10]:
# Save the model to a file
joblib_file = "best_dt.joblib"
joblib.dump(best_dt_classifier, joblib_file)

['best_dt.joblib']

KNN


In [11]:
from imblearn.over_sampling import BorderlineSMOTE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Assume X_final and y_encoded are your features and target after preprocessing


# Apply StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply Borderline SMOTE to the training data only
smote = BorderlineSMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Define the parameter grid for GridSearchCV
param_grid = {
    'n_neighbors': [1, 2, 3, 4, 5]
}

# Initialize the KNN classifier
knn_classifier = KNeighborsClassifier()

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=knn_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV with resampled data
grid_search.fit(X_train_resampled, y_train_resampled)

# Get the best model
best_knn_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_knn = best_knn_classifier.predict(X_test_scaled)

# Evaluate the best model
accuracy_best_knn = accuracy_score(y_test, y_pred_best_knn)
classification_report_result_best_knn = classification_report(y_test, y_pred_best_knn, digits=3)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy (Best KNN with Borderline SMOTE): {accuracy_best_knn}")
print("Classification Report (Best KNN with Borderline SMOTE):\n", classification_report_result_best_knn)


Fitting 5 folds for each of 5 candidates, totalling 25 fits
Best Parameters: {'n_neighbors': 1}
Accuracy (Best KNN with Borderline SMOTE): 0.9970845481049563
Classification Report (Best KNN with Borderline SMOTE):
               precision    recall  f1-score   support

           0      0.974     1.000     0.987        75
           1      1.000     1.000     1.000        60
           2      0.995     1.000     0.997       182
           3      1.000     1.000     1.000        37
           4      1.000     0.995     0.998       212
           5      1.000     1.000     1.000        77
           6      0.996     1.000     0.998       248
           7      1.000     1.000     1.000        41
           8      1.000     1.000     1.000        58
           9      1.000     0.988     0.994       167
          10      1.000     1.000     1.000        43
          11      1.000     0.964     0.982        28
          12      1.000     1.000     1.000        66
          13      1.000     

In [12]:
# Save the model to a file
joblib_file = "best_knn.joblib"
joblib.dump(best_knn_classifier, joblib_file)

['best_knn.joblib']

SVM

In [13]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import BorderlineSMOTE

# Assuming X_train, X_test, y_train, y_test are already defined
# Apply StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply BorderlineSMOTE
borderline_smote = BorderlineSMOTE(random_state=42)
X_train_resampled, y_train_resampled = borderline_smote.fit_resample(X_train_scaled, y_train)

# Define the parameter grid for SVM
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
}

# Initialize the SVM classifier
svm_classifier = SVC()

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=svm_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled, y_train_resampled)

# Get the best model
best_svm_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_svm = best_svm_classifier.predict(X_test_scaled)

# Evaluate the best model
accuracy_best_svm = accuracy_score(y_test, y_pred_best_svm)
classification_report_result_best_svm = classification_report(y_test, y_pred_best_svm, digits=3)

# Print results
print(f"Best Parameters (SVM with BorderlineSMOTE): {grid_search.best_params_}")
print(f"Accuracy (SVM with BorderlineSMOTE): {accuracy_best_svm}")
print("Classification Report (SVM with BorderlineSMOTE):\n", classification_report_result_best_svm)


Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best Parameters (SVM with BorderlineSMOTE): {'C': 100, 'gamma': 0.001}
Accuracy (SVM with BorderlineSMOTE): 0.9967201166180758
Classification Report (SVM with BorderlineSMOTE):
               precision    recall  f1-score   support

           0      0.974     1.000     0.987        75
           1      0.984     1.000     0.992        60
           2      0.995     1.000     0.997       182
           3      1.000     1.000     1.000        37
           4      1.000     0.995     0.998       212
           5      1.000     1.000     1.000        77
           6      0.996     1.000     0.998       248
           7      1.000     1.000     1.000        41
           8      1.000     1.000     1.000        58
           9      1.000     0.988     0.994       167
          10      1.000     1.000     1.000        43
          11      1.000     0.929     0.963        28
          12      1.000     1.000     1.000        66
    

In [14]:
# Save the model to a file
joblib_file = "best_svm.joblib"
joblib.dump(best_svm_classifier, joblib_file)

['best_svm.joblib']

LR

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import BorderlineSMOTE

# Assuming X_train, X_test, y_train, y_test are already defined
# Apply StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply BorderlineSMOTE
smote = BorderlineSMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Define the parameter grid for Logistic Regression
param_grid = {
    'C': [0.1, 1, 10, 100],
    'solver': ['lbfgs', 'liblinear'],  # Choose either 'lbfgs' or 'liblinear' solver
}

# Initialize the Logistic Regression classifier
lr_classifier = LogisticRegression(max_iter=1000)  # Increase max_iter if necessary

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=lr_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled, y_train_resampled)

# Get the best model
best_lr_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_lr = best_lr_classifier.predict(X_test_scaled)

# Evaluate the best model
accuracy_best_lr = accuracy_score(y_test, y_pred_best_lr)
classification_report_result_best_lr = classification_report(y_test, y_pred_best_lr, digits=3)

# Print results
print(f"Best Parameters (Logistic Regression with BorderlineSMOTE): {grid_search.best_params_}")
print(f"Accuracy (Logistic Regression with BorderlineSMOTE): {accuracy_best_lr}")
print("Classification Report (Logistic Regression with BorderlineSMOTE):\n", classification_report_result_best_lr)


Fitting 5 folds for each of 8 candidates, totalling 40 fits
Best Parameters (Logistic Regression with BorderlineSMOTE): {'C': 0.1, 'solver': 'lbfgs'}
Accuracy (Logistic Regression with BorderlineSMOTE): 0.9963556851311953
Classification Report (Logistic Regression with BorderlineSMOTE):
               precision    recall  f1-score   support

           0      0.974     1.000     0.987        75
           1      0.983     0.983     0.983        60
           2      1.000     1.000     1.000       182
           3      1.000     1.000     1.000        37
           4      1.000     1.000     1.000       212
           5      1.000     1.000     1.000        77
           6      1.000     1.000     1.000       248
           7      1.000     1.000     1.000        41
           8      1.000     1.000     1.000        58
           9      1.000     0.988     0.994       167
          10      1.000     1.000     1.000        43
          11      1.000     0.964     0.982        28
        

In [16]:
joblib_file = "best_lr.joblib"
joblib.dump(best_lr_classifier, joblib_file)

['best_lr.joblib']

XGBoost

In [4]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')  # Ignore warnings for simplicity

# Define the parameter grid for XGBoost
param_grid = {
    'max_depth': [2, 3, 4, 5, None],
    'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, None],
}

# Initialize the XGBoost classifier
xgb_classifier = XGBClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=xgb_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled_borderline, y_train_resampled_borderline)

# Get the best model
best_xgb_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_xgb = best_xgb_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_xgb = accuracy_score(y_test, y_pred_best_xgb)
classification_report_result_best_xgb = classification_report(y_test, y_pred_best_xgb, digits=3)

# Print results
print(f"Best Parameters (XGBoost with SMOTE): {grid_search.best_params_}")
print(f"Accuracy (XGBoost with SMOTE): {accuracy_best_xgb}")
print("Classification Report (XGBoost with SMOTE):\n", classification_report_result_best_xgb)


Fitting 5 folds for each of 30 candidates, totalling 150 fits
Best Parameters (XGBoost with SMOTE): {'gamma': None, 'max_depth': 4}
Accuracy (XGBoost with SMOTE): 0.9938046647230321
Classification Report (XGBoost with SMOTE):
               precision    recall  f1-score   support

           0      0.974     1.000     0.987        75
           1      0.967     0.967     0.967        60
           2      1.000     1.000     1.000       182
           3      1.000     1.000     1.000        37
           4      0.991     0.995     0.993       212
           5      1.000     1.000     1.000        77
           6      0.996     0.996     0.996       248
           7      1.000     1.000     1.000        41
           8      0.983     1.000     0.991        58
           9      1.000     0.988     0.994       167
          10      1.000     1.000     1.000        43
          11      1.000     0.857     0.923        28
          12      1.000     1.000     1.000        66
          13    

In [5]:
import joblib
# Save the model to a file
joblib_file = "best_xgb.joblib"
joblib.dump(best_xgb_classifier, joblib_file)

['best_xgb.joblib']

AdaBoost

In [6]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')  # Ignore warnings for simplicity


# Define the parameter grid for AdaBoost
param_grid = {
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.001, 0.1, 1],
}

# Initialize the AdaBoost classifier
ada_classifier = AdaBoostClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=ada_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV on resampled data
grid_search.fit(X_train_resampled_borderline, y_train_resampled_borderline)

# Get the best model
best_ada_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_ada = best_ada_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_ada = accuracy_score(y_test, y_pred_best_ada)
classification_report_result_best_ada = classification_report(y_test, y_pred_best_ada, digits=3)

# Print results
print(f"Best Parameters (AdaBoost with SMOTE): {grid_search.best_params_}")
print(f"Accuracy (AdaBoost with SMOTE): {accuracy_best_ada}")
print("Classification Report (AdaBoost with SMOTE):\n", classification_report_result_best_ada)


Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best Parameters (AdaBoost with SMOTE): {'learning_rate': 0.1, 'n_estimators': 200}
Accuracy (AdaBoost with SMOTE): 0.4293002915451895
Classification Report (AdaBoost with SMOTE):
               precision    recall  f1-score   support

           0      0.785     0.827     0.805        75
           1      0.310     0.650     0.419        60
           2      0.427     0.192     0.265       182
           3      0.000     0.000     0.000        37
           4      0.243     0.599     0.346       212
           5      1.000     0.065     0.122        77
           6      0.692     0.290     0.409       248
           7      0.000     0.000     0.000        41
           8      0.846     0.190     0.310        58
           9      1.000     0.078     0.144       167
          10      0.000     0.000     0.000        43
          11      1.000     0.107     0.194        28
          12      0.976     0.606     0.748        66
  

In [7]:
import joblib
# Save the model to a file
joblib_file = "best_ada.joblib"
joblib.dump(best_ada_classifier, joblib_file)

['best_ada.joblib']

CatBoost

LGBM

In [8]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')  # Ignore warnings for simplicity

# Define the parameter grid for LGBM
param_grid = {
    'max_depth': [-1, 2, 3, 4, 5],
    'n_estimators': [50, 100, 150, 200],
}

# Initialize the LGBM classifier
lgbm_classifier = LGBMClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=lgbm_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Assuming X_train_resampled_smote, y_train_resampled_smote are defined
# Fit GridSearchCV on SMOTE resampled data
grid_search.fit(X_train_resampled_borderline, y_train_resampled_borderline)

# Get the best model
best_lgbm_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_lgbm = best_lgbm_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_lgbm = accuracy_score(y_test, y_pred_best_lgbm)
classification_report_result_best_lgbm = classification_report(y_test, y_pred_best_lgbm, digits=3)

# Print results
print(f"Best Parameters (LGBM with SMOTE): {grid_search.best_params_}")
print(f"Accuracy (LGBM with SMOTE): {accuracy_best_lgbm}")
print("Classification Report (LGBM with SMOTE):\n", classification_report_result_best_lgbm)


Fitting 5 folds for each of 20 candidates, totalling 100 fits
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.038295 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 97920
[LightGBM] [Info] Number of data points in the train set: 37308, number of used features: 384
[LightGBM] [Info] Start training from score -3.003482
[LightGBM] [Info] Start training from score -3.003482
[LightGBM] [Info] Start training from score -3.003482
[LightGBM] [Info] Start training from score -3.003482
[LightGBM] [Info] Start training from score -3.003482
[LightGBM] [Info] Start training from score -3.003482
[LightGBM] [Info] Start training from score -3.003482
[LightGBM] [Info] Start training from score -5.629123
[LightGBM] [Info] Start training from score -3.003482
[LightGBM] [Info] Start training from score -3.003482
[LightGBM] [Info] Start training from score -5.490010
[LightGBM] [Info] Start training from score -3.003482

In [9]:
joblib_file = "best_lgbm_classifierMpnet.joblib"
joblib.dump(best_lgbm_classifier, joblib_file)

['best_lgbm_classifierMpnet.joblib']